<h3>Transformadas de Fourrier</h3>

Neste trecho faremos um teste das bibliotecas para transformar o meu mp3 em uma transformada de Fourrier, e se é mais viável compuacionalmente fazer a transformada ao mesmo tempo do download, ou se é mais viável fazer todo o download, e então transformar.

In [1]:
import librosa
import numpy as np
import matplotlib.pyplot as plt
import librosa.display

# Carrega o áudio
audio, sr = librosa.load("../dataset/avoante/avoante_10.mp3", sr=None)

stft = librosa.stft(
    audio,
    n_fft=2048,      # tamanho da FFT
    hop_length=512   # avanço da janela
)

stft_mag = np.abs(stft)
stft_db = librosa.amplitude_to_db(stft_mag, ref=np.max)

plt.figure(figsize=(10, 4))
librosa.display.specshow(
    stft_db,
    sr=sr,
    hop_length=512,
    x_axis="time",
    y_axis="hz"
)
plt.colorbar(format="%+2.0f dB")
plt.title("Espectrograma (STFT)")
plt.tight_layout()
plt.show()


C:\Users\Miguel\AppData\Local\Temp\ipykernel_17860\3595488343.py:7: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, sr = librosa.load("../dataset/avoante/avoante_10.mp3", sr=None)
c:\Users\Miguel\anaconda3\envs\gpu2\lib\site-packages\librosa\core\audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


FileNotFoundError: [Errno 2] No such file or directory: '../dataset/avoante/avoante_10.mp3'

Código de teste, vou criar uma outra pasta do avoante, porém com as STFT armazenadas ao invés do aúdio bruto

In [2]:
import requests
from bs4 import BeautifulSoup
import inspect
import json
import os   
from pathlib import Path
import pandas as pd
import re
import io

headers = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/121.0.0.0 Safari/537.36"
    ),
    "Accept-Language": "pt-BR,pt;q=0.9",
    "Referer": "https://www.wikiaves.com.br/"
}


In [3]:
#uma função idêntica à anterior, no interior ela salva o espectro ao invés do mp3

def createDataset(codigoAve, nomeAve, dwldLimit):

    urlFiles = "https://www.wikiaves.com.br//getRegistrosJSON.php?tm=s&t=s&s="+str(codigoAve)+"&o=mp&o=mp&p=1"

    #Essa url chama o endpoint /getRegistrosJSON.php
    #Os seus parâmetros, não conheço os detalhes de todos ainda
    #no entanto, eu sei que tm=s significa tipo mídia = som

    responseFiles = requests.get(urlFiles, headers=headers)

    cont = 0
    data = json.loads(responseFiles.text)["registros"]["itens"]

    if not Path("../dataset/"+nomeAve+"Espec").exists(): os.makedirs("../dataset/"+nomeAve+"Espec")

    for registro in data:
        link = data[registro]["link"]

        #ainda não descobri como pegar diretamente o link dos sons em mp3,
        #ao invés dos espectogramas diretamente, por isso, ao substituir
        #na url mp3 por jpg, podemos pegar os sons brutos
        link = link.replace(".jpg", ".mp3")

        #por algum motivo, os links do s3 bucket com # são protegisdos,
        #e sem o # são totalmente livres
        link = link.replace("#", "")

        try:
            responseAudio = requests.get(link, timeout=10)

            if responseAudio.status_code != 200:
                continue

            content_type = responseAudio.headers.get("Content-Type", "")
            if not content_type.startswith("audio"):
                continue

            if len(responseAudio.content) < 10_000:  # ~10 KB
                continue

            # tenta carregar
            audio, sr = librosa.load(
                io.BytesIO(responseAudio.content),
                sr=None
            )
        except Exception as e:
            # log simples e segue o baile
            print(f"Erro no arquivo {registro}: {e}")
            continue

        stft = librosa.stft(
            audio,
            n_fft=2048,      # tamanho da FFT
            hop_length=512   # avanço da janela
        )

        stft_mag = np.abs(stft) 
        stft_db = librosa.amplitude_to_db(stft_mag, ref=np.max)
        path = Path("..") / "dataset" / f"{nomeAve}Espec" / f"{nomeAve}_{cont}.jpg"
        
        np.save(path.with_suffix(".npy"), stft_db)

        #with open(path) as f:
        #    f.write(stft_db)

        if cont >= dwldLimit:
            break

        cont+=1

In [5]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# pasta onde estão os .npy
DATASET_DIR = Path("../dataset/avoanteEspec")

# lista todos os arquivos .npy
files = sorted(DATASET_DIR.glob("*.npy"))

for file in files:
    spec = np.load(file)

    plt.figure(figsize=(10, 4))
    librosa.display.specshow(
        spec,
        sr=sr,
        hop_length=512,
        x_axis="time",
        y_axis="hz"
    )
    plt.colorbar(format="%+2.0f dB")
    plt.title("Espectrograma (STFT)")
    plt.tight_layout()
    plt.show()



NameError: name 'sr' is not defined

<Figure size 1000x400 with 0 Axes>

In [ ]:
#uma função idêntica à anterior, no interior ela salva o espectro ao invés do mp3

def createDataset(codigoAve, nomeAve, dwldLimit):
    #Essa url chama o endpoint /getRegistrosJSON.php
    #Os seus parâmetros, não conheço os detalhes de todos ainda
    #no entanto, eu sei que tm=s significa tipo mídia = som

    urlFiles = "https://www.wikiaves.com.br//getRegistrosJSON.php?tm=s&t=s&s="+str(codigoAve)+"&o=mp&o=mp&p=1"
    headers = {
        "User-Agent": (
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
            "AppleWebKit/537.36 (KHTML, like Gecko) "
            "Chrome/121.0.0.0 Safari/537.36"
        ),
        "Accept-Language": "pt-BR,pt;q=0.9",
        "Referer": "https://www.wikiaves.com.br/"
    }

    responseFiles = requests.get(urlFiles, headers=headers)

    TARGET_SECONDS = 25
    cont = 0
    saved = 0

    data = json.loads(responseFiles.text)["registros"]["itens"]

    if not Path("../dataset/"+nomeAve+"Espec").exists(): os.makedirs("../dataset/"+nomeAve+"Espec")

    for registro in data:
        link = data[registro]["link"]

        #ainda não descobri como pegar diretamente o link dos sons em mp3,
        #ao invés dos espectogramas diretamente, por isso, ao substituir
        #na url mp3 por jpg, podemos pegar os sons brutos
        link = link.replace(".jpg", ".mp3")

        #por algum motivo, os links do s3 bucket com # são protegisdos,
        #e sem o # são totalmente livres
        link = link.replace("#", "")

        #Precisamos de um try, pois alguns downloads tiveram problemas 
        try:
            responseAudio = requests.get(link, timeout=10)
            audio, sr = librosa.load(
                io.BytesIO(responseAudio.content),
                sr=None
            )

            target_samples = TARGET_SECONDS * sr

            if len(audio) < target_samples:
                pad = target_samples - len(audio)
                audio = librosa.util.fix_length(audio, size=target_samples)
            else:
                audio = audio[:target_samples]
                
        except Exception as e:
            print(f"Erro no arquivo {registro}: {e}")
            continue

        stft = librosa.stft(
            audio,
            n_fft=2048,      # tamanho da FFT
            hop_length=512   # avanço da janela
        )

        stft_mag = np.abs(stft) 
        stft_db = librosa.amplitude_to_db(stft_mag, ref=np.max)
        path = Path("..") / "dataset" / f"{nomeAve}Espec" / f"{nomeAve}_{cont}.jpg"
        
        np.save(path.with_suffix(".npy"), stft_db)

        if cont >= dwldLimit:
            break

        cont+=1
